This notebook is for data preparation of your csv file which presumably was downloaded from your home banking software. 

First we need to install the libraries we are going to use. 

In [ ]:
 !pip install openai pandas

set up the program and load the libraries 

In [ ]:
import pandas as pd
import openai
import time

openai.api_key = ""

Below is loading the data. nrows limits the amount of data that gets loaded. You can change the name and path to the file as needed. 

In [ ]:
# Load the financial data from a CSV file
data = pd.read_csv("data.csv",nrows=1500)

Define the OpenAI model and prompt for categorization

In [ ]:
# Define the OpenAI model and prompt for categorization
model = "text-davinci-003"
prompt_intro = "The following is a financial transaction description and the spending categories they fall into:"
start_sequence = "please respond with category equals \n"
number_rows = 1500

In [ ]:


# Initialize validation statistics variables
total = 0

# Define a list of descriptions to categorize
descriptions = list(data["Description"][:number_rows])
print(descriptions)

# Use OpenAI's API to categorize the descriptions
categories = []

for description in descriptions:
    final_prompt = prompt_intro + description + start_sequence
    response = None
    while response is None:
        try:
            response = openai.Completion.create(
                engine=model,
                prompt=final_prompt,
                max_tokens=1024,
                n=1,
                stop=None,
                temperature=0.5,
            )
        except openai.error.APIError as e:
            print(f"API request failed: {e}")
            print("Retrying in 1 minute...")
            time.sleep(60)

    for choice in response.choices:
        response_text = choice.text.strip()
        print(f"the response_text is {response_text}")
        if "Category:" in response_text:
            # Extract the category from the "Category:" line and remove the prefix
            category = response_text.split("Category:")[1].strip()
            category = category.replace(",", "")  # Remove any commas in the category string
        elif "Transfer" in response_text:
        # Set the category to "Transfer Fees" if the response contains this string
            category = "Transfer"
        elif "Category =" in response_text:
        # Extract the category from the "Category:" line and remove the prefix
            category = response_text.split("Category =")[1].strip()
            category = category.replace(",", "")  # Remove any commas in the category string
        elif "Category=" in response_text:
        # Extract the category from the "Category:" line and remove the prefix
            category = response_text.split("Category=")[1].strip()
            category = category.replace(",", "")  # Remove any commas in the category string
        else:
            # Set the category to "Unknown" if the pattern doesn't match
            category = response_text
    print(category)
    total += 1
    print(f"total={total}")
    categories.append(category)

    # Introduce a delay of 1 second between API requests
    time.sleep(.5)

# Add the categories to the data DataFrame
#data.loc["Category"] = categories 
data.loc[:number_rows-1, "Category"] = categories 

# Write the updated data to a new CSV file
data.to_csv("data_cat2.csv", index=False)